In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
anime = pd.read_csv('/Users/z2271499/Downloads/anime/anime.csv')
anime

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17557,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",Unknown,盗墓笔记之秦岭神树,ONA,Unknown,"Apr 4, 2021 to ?",Unknown,...,Unknown,Unknown,Unknown,1.0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
17558,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",Unknown,見える子ちゃん,TV,Unknown,2021 to ?,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
17559,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Higurashi:When They Cry – SOTSU,ひぐらしのなく頃に卒,TV,Unknown,"Jul, 2021 to ?",Summer 2021,...,1.0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
17560,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",Unknown,ヤマノススメ Next Summit,TV,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown


In [15]:
# get the columns we want
anime = anime[['Name', 'Genres', 'Type', 'Score', 'Ranked']]
anime

,Name,Genres,Type,Score,Ranked
0,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,8.78,28.0
1,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",Movie,8.39,159.0
2,Trigun,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",TV,8.24,266.0
3,Witch Hunter Robin,"Action, Mystery, Police, Supernatural, Drama, ...",TV,7.27,2481.0
4,Bouken Ou Beet,"Adventure, Fantasy, Shounen, Supernatural",TV,6.98,3710.0
...,...,...,...,...,...
17557,Daomu Biji Zhi Qinling Shen Shu,"Adventure, Mystery, Supernatural",ONA,Unknown,Unknown
17558,Mieruko-chan,"Comedy, Horror, Supernatural",TV,Unknown,Unknown
17559,Higurashi no Naku Koro ni Sotsu,"Mystery, Dementia, Horror, Psychological, Supe...",TV,Unknown,Unknown
17560,Yama no Susume: Next Summit,"Adventure, Slice of Life, Comedy",TV,Unknown,Unknown


In [16]:
anime = anime[anime['Score'] != 'Unknown']
anime = anime[anime['Ranked'] != 'Unknown']
anime

,Name,Genres,Type,Score,Ranked
0,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,8.78,28.0
1,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",Movie,8.39,159.0
2,Trigun,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",TV,8.24,266.0
3,Witch Hunter Robin,"Action, Mystery, Police, Supernatural, Drama, ...",TV,7.27,2481.0
4,Bouken Ou Beet,"Adventure, Fantasy, Shounen, Supernatural",TV,6.98,3710.0
...,...,...,...,...,...
17504,Kimetsu Gakuen: Valentine-hen,Comedy,ONA,6.59,5461.0
17505,Heikousen,"Music, Romance",Music,7.52,1517.0
17512,Nu Wushen de Canzhuo Spring Festival Special,"Slice of Life, Comedy",Special,6.83,4382.0
17513,Yakusoku no Neverland 2nd Season: Michishirube,"Mystery, Psychological, Supernatural, Thriller...",Special,4.81,10760.0


In [17]:
anime = anime.join(anime['Genres'].str.get_dummies(','))
anime

,Name,Genres,Type,Score,Ranked,Adventure,Cars,Comedy,Dementia,Demons,...,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire
0,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,8.78,28.0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",Movie,8.39,159.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Trigun,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",TV,8.24,266.0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Witch Hunter Robin,"Action, Mystery, Police, Supernatural, Drama, ...",TV,7.27,2481.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bouken Ou Beet,"Adventure, Fantasy, Shounen, Supernatural",TV,6.98,3710.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17504,Kimetsu Gakuen: Valentine-hen,Comedy,ONA,6.59,5461.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17505,Heikousen,"Music, Romance",Music,7.52,1517.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17512,Nu Wushen de Canzhuo Spring Festival Special,"Slice of Life, Comedy",Special,6.83,4382.0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
17513,Yakusoku no Neverland 2nd Season: Michishirube,"Mystery, Psychological, Supernatural, Thriller...",Special,4.81,10760.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# vectorize the data
vectorizer = CountVectorizer()
vectorized = vectorizer.fit_transform(anime)

NameError: name 'item_' is not defined